In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Activation
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
# Be sure it is a jpg

In [ ]:
image_path = "IMAGE_NAME_HERE" # Uploaded image name

In [ ]:
img=mpimg.imread(image_path) # Reading in the image as a matrix of values
plt.imshow(img)

In [ ]:
print(img.shape)

In [ ]:
print(img)

In [ ]:
def nice_plot(model, img):
    img_batch = np.expand_dims(img, axis=0)
    conv_img2 = model.predict(img_batch)
    conv_img2 = np.squeeze(conv_img2, axis=0)
    print(conv_img2.shape)
    plt.imshow(conv_img2)

In [ ]:
def visualize_first_filter(model):
    top_layer = model.layers[0]
    plt.imshow(top_layer.get_weights()[0][:, :, :, 0].squeeze())
    plt.show()

In [ ]:
def nicer_plot(model, img):
    img_batch = np.expand_dims(img, axis=0)
    conv_img2 = model.predict(img_batch)
    conv_img2 = np.squeeze(conv_img2, axis=0)
    print(conv_img2.shape[:2])
    conv_img2 = conv_img2.reshape(conv_img2.shape[:2])
    plt.imshow(conv_img2)

In [ ]:
# Fake model 1
model = Sequential() # Going to have sequential layers
model.add(Convolution2D(3, # Number of filters
                        (5, 5), # Size of the filters
                        input_shape = img.shape)) # Initial image size

# Visualize the first filter
visualize_first_filter(model)

#Visualize what the picture looks after applying the filter
nicer_plot(model, img)

In [ ]:
# Fake model 2
model = Sequential()
model.add(Convolution2D(1, # Number of filters
                        (3, 3), # Size of the filters
                        input_shape = img.shape)) # Initial image size

# Visualize the first filter
visualize_first_filter(model)

#Visualize what the picture looks after applying the filter
nicer_plot(model, img)

In [ ]:
# Fake model 3
model = Sequential()
model.add(Convolution2D(1, # Number of filters
                        (3, 3), # Size of the filters
                        input_shape = img.shape)) # Initial image size
model.add(Activation('relu')) # Removes 

# Visualize the first filter
visualize_first_filter(model)

#Visualize what the picture looks after applying the filter
nicer_plot(model, img)

In [ ]:
# Fake model 4
model = Sequential()
model.add(Convolution2D(1, # Number of filters
                        (3, 3), # Size of the filters
                        input_shape = img.shape)) # Initial image size
model.add(MaxPooling2D(pool_size=(2,2)))

# Visualize the first filter
visualize_first_filter(model)

#Visualize what the picture looks after applying the filter
nicer_plot(model, img)

In [ ]:
# Here down shamelessly taken from https://www.tensorflow.org/tutorials/image_retraining
def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

def load_labels(label_file):
    label = []
    file = open(label_file, "r") 
    proto_as_ascii_lines = file.readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

In [ ]:
def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    
    # Decode the image into an array
    file_reader = tf.read_file(file_name, input_name)
    if file_name.endswith(".png"):
        image_reader = tf.image.decode_png(
        file_reader, channels=3, name="png_reader")
    elif file_name.endswith(".gif"):
        image_reader = tf.squeeze(
        tf.image.decode_gif(file_reader, name="gif_reader"))
    elif file_name.endswith(".bmp"):
        image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
    else:
        image_reader = tf.image.decode_jpeg(
        file_reader, channels=3, name="jpeg_reader")
        
    float_caster = tf.cast(image_reader, tf.float32) # Make sure all numbers are floats
    dims_expander = tf.expand_dims(float_caster, 0) # Extra dimension for wiggle room
    resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width]) # resize
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std]) # Ensure number % of 255
    sess = tf.Session() # Start a tensorflow session
    result = sess.run(normalized) # Get the normalized tensor that tensorflow can read

    return result

In [ ]:
# Already compiled
graph = load_graph('output_graph.pb')
label_file = load_labels('output_labels.txt')
input_layer = "Placeholder"
output_layer = "final_result"

# Set image size for model
input_height = 299
input_width = 299
input_mean = 0
input_std = 255

t = read_tensor_from_image_file(
  image_path,
  input_height=input_height,
  input_width=input_width,
  input_mean=input_mean,
  input_std=input_std)

# gather layers to input and receive image
input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name)
output_operation = graph.get_operation_by_name(output_name)

In [ ]:
# Start a session with the graph set as the model.
# Get model outputs and use these to compare the model
with tf.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
    results = np.squeeze(results)
    top_k = results.argsort()[-5:][::-1] # Order of the top classes
    labels = label_file
    
    for i in top_k:
        print(labels[i], results[i])      
